# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [55]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from scipy.stats import linregress

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [75]:
final=pd.read_csv('final_weather_data.csv')
final

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,klaksvik,100,FO,1584721394,69,62.23,-6.59,277.15,7.70
1,sinnamary,98,GF,1584721394,76,5.38,-52.95,299.90,4.88
2,saint george,90,US,1584721311,87,37.10,-113.58,279.26,2.10
3,ignatovka,42,RU,1584721395,76,53.95,47.65,275.31,4.90
4,hermanus,0,ZA,1584721395,81,-34.42,19.23,293.71,2.24
...,...,...,...,...,...,...,...,...,...
552,channel-port aux basques,100,CA,1584721429,86,47.57,-59.14,273.40,11.94
553,zemaiciu naumiestis,33,LT,1584721430,74,55.36,21.70,278.61,5.39
554,cayenne,52,GF,1584721430,83,4.93,-52.33,300.15,4.60
555,sarahan,65,IN,1584721430,76,31.52,77.80,270.01,2.65


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [57]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557 entries, 0 to 556
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        557 non-null    object 
 1   Cloudiness  557 non-null    int64  
 2   Country     554 non-null    object 
 3   Date        557 non-null    int64  
 4   Humidity    557 non-null    int64  
 5   Lat         557 non-null    float64
 6   Lng         557 non-null    float64
 7   Max Temp    557 non-null    float64
 8   Wind Speed  557 non-null    float64
dtypes: float64(4), int64(3), object(2)
memory usage: 39.3+ KB


In [65]:
fig = gmaps.figure()
locations = final[["Lat","Lng"]].astype(float)
humidity=final["Humidity"]/100
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=10,point_radius = 5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [80]:
final_2=final.loc[(final['Max Temp']>260) & 
                  (final['Max Temp']<280)& 
                  (final['Wind Speed']>10) &
                  (final['Cloudiness']<100)]
final_2

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
12,ushuaia,75,AR,1584721384,81,-54.80,-68.30,279.15,10.30
29,sorland,75,NO,1584721399,79,67.67,12.69,272.15,14.90
51,barrow,90,US,1584721205,92,71.29,-156.79,271.15,12.30
176,glace bay,90,CA,1584721407,93,46.20,-59.96,278.71,11.80
189,tasiilaq,90,GL,1584721408,92,65.61,-37.64,268.15,10.30
305,mehamn,75,NO,1584721415,62,71.04,27.85,269.15,10.30
340,grindavik,90,IS,1584721417,93,63.84,-22.43,278.15,14.40
359,vestmannaeyjar,75,IS,1584721418,81,63.44,-20.27,279.15,14.40
389,kurilsk,99,RU,1584721420,92,45.23,147.88,273.51,10.49
495,kabelvag,75,NO,1584721426,63,68.21,14.48,272.04,10.30


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [111]:
hotel_df=[]
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?types=hotel&radius=5000&location="

for index, row in final_2.iterrows():
    lat = row['Lat']
    lng=row['Lng']
    location=(lat,lng)
    params['location'] = location
    url=base_url + str(lat)+','+str(lng)+'&key='+g_key
    response = requests.get(url).json()
    results = response['results']
    
    try:
        hotel_df.append(results[0]['name']) 
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


final_2=pd.DataFrame(final_2)
final_2["Hotel Name"]=hotel_df
final_2

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,HotelName,hotel_df,Hotel Name
12,ushuaia,75,AR,1584721384,81,-54.80,-68.30,279.15,10.30,Ushuaia,Ushuaia,Ushuaia
29,sorland,75,NO,1584721399,79,67.67,12.69,272.15,14.90,Sørland,Sørland,Sørland
51,barrow,90,US,1584721205,92,71.29,-156.79,271.15,12.30,Utqiagvik,Utqiagvik,Utqiagvik
176,glace bay,90,CA,1584721407,93,46.20,-59.96,278.71,11.80,Glace Bay,Glace Bay,Glace Bay
189,tasiilaq,90,GL,1584721408,92,65.61,-37.64,268.15,10.30,Tasiilaq,Tasiilaq,Tasiilaq
305,mehamn,75,NO,1584721415,62,71.04,27.85,269.15,10.30,Mehamn,Mehamn,Mehamn
340,grindavik,90,IS,1584721417,93,63.84,-22.43,278.15,14.40,Grindavik,Grindavik,Grindavik
359,vestmannaeyjar,75,IS,1584721418,81,63.44,-20.27,279.15,14.40,Vestmannaeyjar,Vestmannaeyjar,Vestmannaeyjar
389,kurilsk,99,RU,1584721420,92,45.23,147.88,273.51,10.49,Kurilsk,Kurilsk,Kurilsk
495,kabelvag,75,NO,1584721426,63,68.21,14.48,272.04,10.30,Svolvær,Svolvær,Svolvær


In [112]:
fig = gmaps.figure()
locations = final_2[["Lat","Lng"]].astype(float)
humidity=final_2["Humidity"]/100
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius = 5)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [114]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in final_2.iterrows()]
#locations = hotel_df[["Lat", "Lng"]]
locations = final_2[["Lat","Lng"]].astype(float)

In [116]:
# Add marker layer ontop of heat map
namelist=final_2['Hotel Name'].tolist()

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Hotel name: {name}" for name in namelist]
)


fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))